In [1]:
import os
try:
    import boto3

    s3 = boto3.resource(
        "s3",
        region_name="us-east-1",
        aws_secret_access_key=os.getenv("AWS_SECRET_AK"),
        aws_access_key_id=os.getenv("AWS_AK"),
    )
    bucket = s3.Bucket("ids703-nlp-finalproject")
    SAVE_TO_S3 = True
    print("[INFO] S3 connection successful.")
except:
    print("[ERROR] Could not connect to S3! Only saving locally.")
    SAVE_TO_S3 = False


ValueError: Record does not exist.

In [ ]:
import optuna
from optuna.visualization import plot_parallel_coordinate
bucket.download_file("artefacts/tf_hyperparameter_study.db", "../artefacts/tf_hyperparameter_study.db")
study = optuna.load_study(study_name="tf_study001", storage="sqlite:///../artefacts/tf_hyperparameter_study.db")
plot_parallel_coordinate(study)